In [2]:
import pandas as pd
import numpy as np

In [66]:
data = pd.read_csv(".../Updated_MS_14000.csv")
data.head()

,Patient_ID,Date_of_visit,Pyramidal,Cerebellar,Thronchioencephalic,Sensitive,Sphincteric,Visual,Mental,Deambulation,EDSS_score_assessed_by_clinician,Age,Sex,MS in pediatric age
0,2774,2013-05-07,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,30,0,0
1,2774,2013-12-04,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,31,0,0
2,2774,2015-04-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,32,0,0
3,2774,2015-06-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,32,0,0
4,2774,2015-11-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,32,0,0


In [83]:
postcode = pd.read_csv("C:/Users/Mahin Vazifehdan/Desktop/Git_Learning/MSSS_on_FSscores/Patient_CAP.csv")
onset = pd.read_csv("C:/Users/Mahin Vazifehdan/Desktop/Git_Learning/MSSS_on_FSscores/Onset_date.csv")
Clinical_data_with_CAP = pd.merge(data,postcode, left_on="Patient_ID", right_on="Paziente ID", how = "left")
Clinical_data_with_CAP = pd.merge(Clinical_data_with_CAP,onset, left_on="Patient_ID", right_on="Patient_id", how = "left")
Clinical_data_with_CAP.drop(["Paziente ID","Patient_id"], axis= 1, inplace = True)
Clinical_data_with_CAP.rename(columns={"Data di nascita": "Date_of_Birth","Onset_date": "Date_of_Onset"}, inplace=True)
Clinical_data_with_CAP.drop(Clinical_data_with_CAP[Clinical_data_with_CAP["CAP"].isnull()]
                            .index, inplace = True)
Clinical_data_with_CAP['Date_of_visit'] = pd.to_datetime(Clinical_data_with_CAP['Date_of_visit'],
                                                         format ='%Y-%m-%d')
Clinical_data_with_CAP['Date_of_Onset'] = pd.to_datetime(Clinical_data_with_CAP['Date_of_Onset'],
                                                         format ='%Y-%m-%d')
Clinical_data_with_CAP["CAP"] = Clinical_data_with_CAP["CAP"].astype(int)

first = Clinical_data_with_CAP.groupby('Patient_ID').head(1)
last = Clinical_data_with_CAP.groupby('Patient_ID').tail(1)
result = pd.concat([first,last]).drop_duplicates().sort_values(by=['Patient_ID','Date_of_visit'])

result = result[['Patient_ID','CAP','Date_of_Birth','Date_of_Onset','Date_of_visit']]
result.head()

,Patient_ID,CAP,Date_of_Birth,Date_of_Onset,Date_of_visit
0,2774,70043,1982-12,2011-02-15,2013-05-07
18,2774,70043,1982-12,2011-02-15,2021-04-14
19,3936,72100,1992-04,2000-06-15,2011-02-25
52,3936,72100,1992-04,2000-06-15,2021-04-29
53,5835,70025,1983-02,2005-05-15,2006-04-20


In [84]:
df_grouped = result.groupby('Patient_ID')['Date_of_visit'].agg(First_Date_of_visit='min',
                                                               Last_Date_of_visit='max').reset_index()
df_grouped.head()

,Patient_ID,First_Date_of_visit,Last_Date_of_visit
0,2774,2013-05-07,2021-04-14
1,3936,2011-02-25,2021-04-29
2,5835,2006-04-20,2018-10-11
3,9988,1997-11-04,2016-12-19
4,10096,2000-01-03,2006-03-13


In [85]:
Clinical_data_with_CAP = pd.merge(result,df_grouped, left_on="Patient_ID", right_on="Patient_ID", how = "left")
result = Clinical_data_with_CAP.groupby('Patient_ID').head(1).reset_index()
result[['Patient_ID','CAP','Date_of_Birth','Date_of_Onset','First_Date_of_visit','Last_Date_of_visit']].to_csv("First_Last_Patient.csv", index = False)